In [1]:
import torch
from pytorch_transformers import BertTokenizer, BertForMaskedLM

In [ ]:
model            = {}
model['GooBERT'] = BertForMaskedLM.from_pretrained('GooBERT')
model['FinBERT'] = BertForMaskedLM.from_pretrained('FinBERT-Prime_128MSL-250K')
model['PreBERT'] = BertForMaskedLM.from_pretrained('FinBERT-Pre2K_128MSL-250K')
model['ComBERT'] = BertForMaskedLM.from_pretrained('FinBERT-Combo_128MSL-250K')

In [67]:
S1 = '[CLS] the company has a fiduciary duty to its shareholders . [SEP]'
S2 = 'one of its many regulatory requirements . [SEP]'

S1 = '[CLS] the volatility caused us to what [SEP]'
S2 = 'one of its many regulatory requirements . [SEP]'

S1 = '[CLS] we have geographically diversified sales [SEP]'
S2 = 'we believe that we hold the worst market position [SEP]'

S1 = '[CLS] we have geographically diversified sales [SEP]'
S2 = 'we believe that we hold the best market position [SEP]'

S1 = '[CLS] ceo total compensation [SEP]'
S2 = 'resulting from foo of this year [SEP]'

# sentence_1 = ('[CLS] we have geographically [MASK] sales [SEP]', 'diversified')
# sentence_2 = ('we believe that we hold the [MASK] market position [SEP]', 'best')

# sentence_1 = ('[CLS] we have geographically [MASK] sales [SEP]', 'diversified')
# sentence_2 = ('we believe that we hold the [MASK] market position [SEP]', 'best')


# S1 = sentence_1[0]
# S2 = sentence_2[0]

tokenizer      = BertTokenizer.from_pretrained('bert-base-uncased')
text           = f'{S1} {S2}'
tokenized_text = tokenizer.tokenize(text)

# MI = [i for i, x in enumerate(tokenized_text) if x == "[MASK]"]
# MI = [tokenized_text.index(sentence_1[1]), tokenized_text.index(sentence_2[1])]

for i, word in enumerate(tokenized_text):
    print("{} {}".format(i, word))
    
print(tokenized_text)


0 [CLS]
1 ceo
2 total
3 compensation
4 [SEP]
5 resulting
6 from
7 foo
8 of
9 this
10 year
11 [SEP]
['[CLS]', 'ceo', 'total', 'compensation', '[SEP]', 'resulting', 'from', 'foo', 'of', 'this', 'year', '[SEP]']


In [68]:
MI = [7]

for i in MI :
    tokenized_text[i] = '[MASK]'

print(tokenized_text)

indexed_tokens   = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids     = [0] * len(tokenizer.tokenize(S1)) + [1] * len(tokenizer.tokenize(S2))

tokens_tensor    = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])


preds  = {}
for m in model:
    with torch.no_grad():
        preds[m] = model[m](tokens_tensor, token_type_ids = segments_tensors)[0]
        
for m in preds:
    tokens = []
    for i in MI:
        predicted_index = torch.argmax(preds[m][0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        tokens.append(predicted_token)

    print(f'{m} : {tokens}')

['[CLS]', 'ceo', 'total', 'compensation', '[SEP]', 'resulting', 'from', '[MASK]', 'of', 'this', 'year', '[SEP]']
GooBERT : ['results']
FinBERT : ['most']
PreBERT : ['growth']
ComBERT : ['operations']


Positive examples have better results in Fin, negative Goo performs better.  Financial documents accentuate the positive?

Finbert feels 

